In [8]:
# Usage guide:

# This document is not kept on record. If the calcualtions are to be kept,
# copy the whole notebook and put it in "quick_calculations" or appropriate
# folders.

In [9]:
# import all the E9 stuff
import logging
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# User defined modules
E9path = Path("C:/", "Users", "ken92", "Documents", "Studies", "E5", "simulation", "E9_simulations")
if str(E9path) not in sys.path:
    sys.path.insert(1, str(E9path))
import E9_fn.E9_constants as E9c
import E9_fn.E9_numbers as E9n
import E9_fn.E9_cooling as cool
# import E9_fn.polarizabilities_calculation as E9pol
from E9_fn import util

# Logging
logpath = '' # '' if not logging to a file
loglevel = logging.INFO
logroot = logging.getLogger()
list(map(logroot.removeHandler, logroot.handlers))
list(map(logroot.removeFilter, logroot.filters))
logging.basicConfig(filename = logpath, level = loglevel)

# Check what species you are using!
Currently want to look at: 87Rb

## Magnetic trap

In [10]:
B_grad_com = 100        # [G/cm] B field gradient after compressing
B_grad_decom = 20       # [G/cm] B field gradient after decompressing
delta_rf_final = -10e6  # [MHz] detuning of RF knife at the end of rf evaporation
Rb87_mF = 2             # mF value

V_grad_decom = (B_grad_decom / 1e4 * 1e2) * E9c.mu_B * E9n.Rb87_5_2S1o2_F2.gF * Rb87_mF # [J/m]
V_grad_decom_uK__mm = V_grad_decom / E9c.k_B / 1e-6 / 1e3
V_grad_com = (B_grad_com / 1e4 * 1e2) * E9c.mu_B * E9n.Rb87_5_2S1o2_F2.gF * Rb87_mF # [J/m]
V_grad_com_uK__mm = V_grad_com / E9c.k_B / 1e-6 / 1e3
V_rf_evap_uK = delta_rf_final / 20e9 / 1e-6

print("V_rf_evap = {:.4f} uK".format(V_rf_evap_uK))
print("V_grad_com = {:.4f} uK/mm".format(V_grad_com_uK__mm))
print("(semiclassical) compressed trap size = {:.4f} mm".format(abs(V_rf_evap_uK) / V_grad_com_uK__mm))
print("V_grad_decom = {:.4f} uK/mm".format(V_grad_decom_uK__mm))
print("(semiclassical) decompressed trap size = {:.4f} mm".format(abs(V_rf_evap_uK) / V_grad_decom_uK__mm))

V_rf_evap = -500.0000 uK
V_grad_com = 671.7138 uK/mm
(semiclassical) compressed trap size = 0.7444 mm
V_grad_decom = 134.3428 uK/mm
(semiclassical) decompressed trap size = 3.7218 mm


## ODTs

In [11]:
pwr_ODT = 6     # [W] ODT power at atoms
w0_ODT = 100e-6  # ODT waist (not using the value I have in E9_constants)
fattening = 1.  # Additional fudge factor to account for being off focus, M2 > 1, etc.

I_ODT_max = E9n.I_from_power(pwr_ODT, w0_ODT * fattening)
V_ODT = E9n.V0_from_I(E9c.gamma_Rb87_D2, (E9c.nu_Rb87_4_2P3o2 + E9c.nu_Rb87_4_2P1o2)/2,
                      E9c.f_lw, I_ODT_max, E9n.Rb87_5_2S1o2_F2.gF, 2)
print("Fattage of the beam = {} (beam waist = {:.2e})".format(fattening, w0_ODT * fattening))
print("V_ODT = {:.4f} uK".format(V_ODT / E9c.k_B / 1e-6))
print("V_ODT = {:.4f} kHz".format(V_ODT / E9c.hnobar / 1000))

Fattage of the beam = 1.0 (beam waist = 1.00e-04)
V_ODT = -52.5240 uK
V_ODT = -1094.4218 kHz


## DMD

In [12]:
# input parameters
lambda_DMD = 638e-9     # wavelength used for DMD potential
pwr_DMD = 1             # DMD power at atoms, in watts
DMD_FoV = 200e-6**2     # field of view (region illuminated) by the DMD

I_at_atom = pwr_DMD / DMD_FoV
f_DMD = E9c.c_light / lambda_DMD
V_DMD = E9n.V0_from_I(E9c.gamma_40K_D2, (E9c.nu_K40_4_2P3o2 + E9c.nu_K40_4_2P1o2)/2,
                      f_DMD, I_at_atom, E9n.K40_4_2S1o2_F9o2.gF, 9/2)
print("V_DMD = {} kHz".format(V_DMD / E9c.hnobar / 1000))

# Polarizibility based calculation
# s_pol = E9pol.alpha_pol(0, lambda_DMD, E9pol.K_4S1o2_LOI, '4S1o2')
# print("V_DMD = {} kHz (from polarizibility)".format(- s_pol * I_at_atom * E9c.Z_0 / 2 / E9c.hnobar / 1000))

V_DMD = 82.09433987605135 kHz


## Lattices

In [24]:
pwr_lat_vert = 3
w0_lat_vert = 112e-6

k0_K40D1 = 2 * np.pi / E9c.lambda_K40_D1
I_lat_vert_max = E9n.I_from_power(pwr_lat_vert * 2, w0_lat_vert)
V_vert_lat = E9n.V0_from_I(E9c.gamma_40K_D2, (E9c.nu_K40_4_2P3o2 + E9c.nu_K40_4_2P1o2)/2,
                           E9c.f_lw, I_lat_vert_max, E9n.K40_4_2S1o2_F9o2.gF, 9/2)
omega0_lat_vert = (2 * np.pi / (E9c.lambda_lw / 2)) * np.sqrt(-V_vert_lat / E9c.m_K40)
a0_vert = np.sqrt(E9c.hbar / E9c.m_K40 / omega0_lat_vert)
lamb_dicke_vert = k0_K40D1 * a0_vert / np.sqrt(2)

# print(I_lat_vert_max)
# print(V_vert_lat)
# print(omega0_lat_vert)
# print(a0_vert)
print(lamb_dicke_vert)
print((0.85**(-2) * 5/3)**(1/4) * 0.17)

304505630.9156161
-4.874405877881955e-28
1012208.7034285272
3.9622697018708896e-08
0.22858998133262806
0.2095084992476854
